<a href="https://colab.research.google.com/github/ayulockin/SwAV-TF/blob/master/Fine_Tuning_on_Flower_Full_Dataset_Supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Setups

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras import models

import matplotlib.pyplot as plt 
import numpy as np
import random
import time
import os

tf.random.set_seed(666)
np.random.seed(666)

tfds.disable_progress_bar()

### W&B - Experiment Tracking

In [2]:
%%capture
!pip install wandb

In [3]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### Dataset gathering and preparation

In [4]:
# Gather Flowers dataset
train_ds, validation_ds = tfds.load(
    "tf_flowers",
    split=["train[:85%]", "train[85%:]"],
    as_supervised=True
)

AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64

@tf.function
def scale_resize_image(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, (224, 224)) # Resizing to highest resolution used while training swav
    return (image, label)

training_ds = (
    train_ds
    .map(scale_resize_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

testing_ds = (
    validation_ds
    .map(scale_resize_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.



Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.0. Subsequent calls will reuse this data.


### ResNet50 base and a custom classification head

In [5]:
def get_training_model(trainable=False):
    inputs = layers.Input(shape=(224, 224, 3))
    EXTRACTOR = tf.keras.applications.ResNet50(weights="imagenet", include_top=False,
        input_shape=(224, 224, 3))
    EXTRACTOR.trainable = trainable
    x = EXTRACTOR(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(5, activation="softmax")(x)
    classifier = models.Model(inputs=inputs, outputs=x)
    
    return classifier

In [6]:
model = get_training_model()
model.summary()

94773248/94765736 [==============================] - 2s 0us/step
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 10,245
Non-trainable params: 23,587,712
_________________________________________________________________


### Callback

In [7]:
# Early Stopping to prevent overfitting
early_stopper = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, verbose=2, restore_best_weights=True)

# Without Augmentation

### Warm Up

In [8]:
# get model and compile
model = get_training_model()

model.compile(loss="sparse_categorical_crossentropy", metrics=["acc"],
                     optimizer='adam')

# initialize wandb run
wandb.init(entity='authors', project='swav-tf')

# train
history = model.fit(training_ds,
                 validation_data=testing_ds,
                 epochs=35,
                 callbacks=[WandbCallback(),
                            early_stopper])

Epoch 1/35
49/49 [==============================] - 12s 248ms/step - loss: 1.5814 - acc: 0.2885 - val_loss: 1.5539 - val_acc: 0.3109
Epoch 2/35
49/49 [==============================] - 11s 219ms/step - loss: 1.5063 - acc: 0.3599 - val_loss: 1.5196 - val_acc: 0.3364
Epoch 3/35
49/49 [==============================] - 11s 216ms/step - loss: 1.4792 - acc: 0.3849 - val_loss: 1.5015 - val_acc: 0.3527
Epoch 4/35
49/49 [==============================] - 11s 215ms/step - loss: 1.4616 - acc: 0.3962 - val_loss: 1.4877 - val_acc: 0.3655
Epoch 5/35
49/49 [==============================] - 11s 216ms/step - loss: 1.4480 - acc: 0.4058 - val_loss: 1.4758 - val_acc: 0.3764
Epoch 6/35
49/49 [==============================] - 11s 216ms/step - loss: 1.4365 - acc: 0.4141 - val_loss: 1.4651 - val_acc: 0.3782
Epoch 7/35
49/49 [==============================] - 11s 217ms/step - loss: 1.4263 - acc: 0.4215 - val_loss: 1.4552 - val_acc: 0.3891
Epoch 8/35
49/49 [==============================] - 11s 219ms/step - 

In [9]:
model.save('warmup.h5')

### Fine tune CNN

In [10]:
# prepare model and compiele
model.layers[1].trainable = True

model.compile(loss="sparse_categorical_crossentropy", metrics=["acc"],
                     optimizer=tf.keras.optimizers.Adam(1e-5))

# initialize wandb run
wandb.init(entity='authors', project='swav-tf')

# train
history = model.fit(training_ds,
                 validation_data=testing_ds,
                 epochs=35,
                 callbacks=[WandbCallback(),
                            early_stopper])

Epoch 1/35
49/49 [==============================] - 34s 688ms/step - loss: 1.6727 - acc: 0.3170 - val_loss: 1.4672 - val_acc: 0.3818
Epoch 2/35
49/49 [==============================] - 33s 665ms/step - loss: 1.3820 - acc: 0.4279 - val_loss: 1.3446 - val_acc: 0.4691
Epoch 3/35
49/49 [==============================] - 33s 670ms/step - loss: 1.2861 - acc: 0.4846 - val_loss: 1.2509 - val_acc: 0.4909
Epoch 4/35
49/49 [==============================] - 33s 674ms/step - loss: 1.2162 - acc: 0.5212 - val_loss: 1.1584 - val_acc: 0.5364
Epoch 5/35
49/49 [==============================] - 33s 674ms/step - loss: 1.1146 - acc: 0.5679 - val_loss: 1.1189 - val_acc: 0.5727
Epoch 6/35
49/49 [==============================] - 33s 677ms/step - loss: 0.9908 - acc: 0.6179 - val_loss: 1.0413 - val_acc: 0.6345
Epoch 7/35
49/49 [==============================] - 32s 652ms/step - loss: 0.9558 - acc: 0.6333 - val_loss: 1.0462 - val_acc: 0.6182
Epoch 8/35
49/49 [==============================] - 33s 682ms/step - 

### Evaluation

In [12]:
loss, acc = model.evaluate(testing_ds)
wandb.log({'Test Accuracy': round(acc*100, 2)})

9/9 [==============================] - 1s 144ms/step - loss: 0.9135 - acc: 0.6673


# Training with Augmentation


### Augmentation

In [13]:
# Configs
CROP_SIZE = 224
MIN_SCALE = 0.5
MAX_SCALE = 1.

# Experimental options
options = tf.data.Options()
options.experimental_optimization.noop_elimination = True
options.experimental_optimization.map_vectorization.enabled = True
options.experimental_optimization.apply_default_optimizations = True
options.experimental_deterministic = False
options.experimental_threading.max_intra_op_parallelism = 1

In [14]:
@tf.function
def scale_image(image, label):
	image = tf.image.convert_image_dtype(image, tf.float32)
	return (image, label)

@tf.function
def random_apply(func, x, p):
	return tf.cond(
		tf.less(tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
				tf.cast(p, tf.float32)),
		lambda: func(x),
		lambda: x)
 
@tf.function
def random_resize_crop(image, label):
  # Conditional resizing
  image = tf.image.resize(image, (260, 260))
  # Get the crop size for given min and max scale
  size = tf.random.uniform(shape=(1,), minval=MIN_SCALE*260,
		          maxval=MAX_SCALE*260, dtype=tf.float32)
  size = tf.cast(size, tf.int32)[0]
  # Get the crop from the image
  crop = tf.image.random_crop(image, (size,size,3))
  crop_resize = tf.image.resize(crop, (CROP_SIZE, CROP_SIZE))
  
  return crop_resize, label

@tf.function
def tie_together(image, label):
  # Scale the pixel values
  image, label = scale_image(image , label)
  # random horizontal flip
  image = random_apply(tf.image.random_flip_left_right, image, p=0.5)
  # Random resized crops
  image, label = random_resize_crop(image, label)
  
  return image, label

In [15]:
trainloader = (
	train_ds
	.shuffle(1024)
	.map(tie_together, num_parallel_calls=AUTO)
	.batch(BATCH_SIZE)
	.prefetch(AUTO)
)

trainloader = trainloader.with_options(options)

### Warmup

In [16]:
# get model and compile
model = get_training_model()

model.compile(loss="sparse_categorical_crossentropy", metrics=["acc"],
                     optimizer='adam')

# initialize wandb run
wandb.init(entity='authors', project='swav-tf')

# train
history = model.fit(trainloader,
                 validation_data=testing_ds,
                 epochs=35,
                 callbacks=[WandbCallback(),
                            early_stopper])

Epoch 1/35
49/49 [==============================] - 11s 232ms/step - loss: 1.5935 - acc: 0.2657 - val_loss: 1.5471 - val_acc: 0.3036
Epoch 2/35
49/49 [==============================] - 11s 217ms/step - loss: 1.5420 - acc: 0.3426 - val_loss: 1.5299 - val_acc: 0.3436
Epoch 3/35
49/49 [==============================] - 11s 219ms/step - loss: 1.5271 - acc: 0.3561 - val_loss: 1.5070 - val_acc: 0.3436
Epoch 4/35
49/49 [==============================] - 11s 219ms/step - loss: 1.5095 - acc: 0.3561 - val_loss: 1.4881 - val_acc: 0.3636
Epoch 5/35
49/49 [==============================] - 11s 222ms/step - loss: 1.5013 - acc: 0.3644 - val_loss: 1.4764 - val_acc: 0.3800
Epoch 6/35
49/49 [==============================] - 11s 221ms/step - loss: 1.4851 - acc: 0.3718 - val_loss: 1.4602 - val_acc: 0.3964
Epoch 7/35
49/49 [==============================] - 10s 210ms/step - loss: 1.4782 - acc: 0.3785 - val_loss: 1.5021 - val_acc: 0.3582
Epoch 8/35
49/49 [==============================] - 11s 215ms/step - 

In [17]:
model.save('warmup_augmentation.h5')

### Fine tune CNN

In [18]:
# prepare model and compiele
model.layers[1].trainable = True

model.compile(loss="sparse_categorical_crossentropy", metrics=["acc"],
                     optimizer=tf.keras.optimizers.Adam(1e-5))

# initialize wandb run
wandb.init(entity='authors', project='swav-tf')

# train
history = model.fit(trainloader,
                 validation_data=testing_ds,
                 epochs=35,
                 callbacks=[WandbCallback(),
                            early_stopper])

Epoch 1/35
49/49 [==============================] - 33s 671ms/step - loss: 1.6167 - acc: 0.2792 - val_loss: 1.5346 - val_acc: 0.3418
Epoch 2/35
49/49 [==============================] - 31s 632ms/step - loss: 1.5041 - acc: 0.3535 - val_loss: 1.7565 - val_acc: 0.1782
Epoch 3/35
49/49 [==============================] - 33s 664ms/step - loss: 1.4996 - acc: 0.3683 - val_loss: 1.3907 - val_acc: 0.4509
Epoch 4/35
49/49 [==============================] - 33s 671ms/step - loss: 1.4181 - acc: 0.4093 - val_loss: 1.3449 - val_acc: 0.4364
Epoch 5/35
49/49 [==============================] - 33s 672ms/step - loss: 1.3706 - acc: 0.4311 - val_loss: 1.3143 - val_acc: 0.4927
Epoch 6/35
49/49 [==============================] - 32s 646ms/step - loss: 1.3665 - acc: 0.4349 - val_loss: 1.3287 - val_acc: 0.4636
Epoch 7/35
49/49 [==============================] - 33s 678ms/step - loss: 1.2956 - acc: 0.4715 - val_loss: 1.2758 - val_acc: 0.4691
Epoch 8/35
49/49 [==============================] - 33s 674ms/step - 

### Evaluation

In [19]:
loss, acc = model.evaluate(testing_ds)
wandb.log({'Test Accuracy': round(acc*100, 2)})

9/9 [==============================] - 1s 146ms/step - loss: 0.7615 - acc: 0.7509
